In [1]:
from datasets import load_dataset
import torch
from tqdm import tqdm

In [2]:
dataset = load_dataset("ag_news")

Using custom data configuration default
Reusing dataset ag_news (/home/travis/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)


In [3]:
print(len(dataset['test']))
dataset['test']['text'][34]

7600


'Explosions Echo Throughout Najaf NAJAF, Iraq - Explosions and gunfire rattled through the city of Najaf as U.S. troops in armored vehicles and tanks rolled back into the streets here Sunday, a day after the collapse of talks - and with them a temporary cease-fire - intended to end the fighting in this holy city...'

In [4]:
from transformers import AutoModel, AutoTokenizer

model_name = "princeton-nlp/sup-simcse-bert-base-uncased"
device = "cuda"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)

In [5]:
V = []
#for row, _ in tqdm(zip(dataset['test'], range(100))):
for row in tqdm(dataset['test']):
    tokens = tokenizer([row['text']], padding=True, truncation=True, return_tensors="pt")
    tokens = tokens.to(device)
    with torch.no_grad():
        embed = model(**tokens, output_hidden_states=True, return_dict=True).pooler_output
    V.append(embed.detach().cpu().numpy().ravel())

100%|████████████████████████████████████████████████| 7600/7600 [01:09<00:00, 109.47it/s]


In [6]:
V[:2]

[array([ 2.85986751e-01, -8.90771374e-02, -4.31362018e-02,  6.34835884e-02,
        -1.16985396e-01, -1.51952103e-01, -8.30738544e-02,  3.56981382e-02,
         3.88802076e-03,  2.40167052e-01, -2.12710481e-02,  1.70543231e-02,
        -1.28378272e-01,  6.95449561e-02, -2.13292941e-01,  1.81594625e-01,
         6.53628260e-02, -1.93467200e-01, -7.59760514e-02,  5.01336344e-03,
         1.65427998e-02, -3.18107679e-02,  1.39391631e-01,  2.96022873e-02,
        -1.86891243e-01, -2.13128631e-04,  7.42615294e-03,  6.60503879e-02,
        -3.36335301e-01, -7.57990479e-02,  2.89555788e-01, -4.46193963e-02,
         9.79716424e-03, -1.78467765e-01, -1.50670961e-01,  8.78205225e-02,
         6.89569919e-04, -7.11075440e-02, -2.37349868e-01, -4.53503989e-02,
         1.10171452e-01,  2.34254426e-03,  5.90086691e-02,  3.96753788e-01,
         1.36969253e-01, -3.93369436e-01,  5.25960959e-02,  8.50704536e-02,
        -8.04332793e-02, -1.07925132e-01, -4.29567605e-01, -2.29109764e-01,
         6.4

In [11]:
import pandas as pd
import numpy as np

df = pd.DataFrame(dataset['test'])
df.to_csv('dataset.csv')
np.save('embeddings.npy', V)

In [12]:
import umap
clf = umap.UMAP(min_dist=0.20)
reduced_embedding = clf.fit_transform(V)

In [15]:
np.save("umap.npy", reduced_embedding)